In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.cm import get_cmap
import pandas as pd
import json
import math
from statsmodels.nonparametric.smoothers_lowess import lowess
# from scipy import loadtxt, optimize
from scipy.signal import argrelmax
from scipy.signal import argrelmin
from scipy import signal
import re
from scipy import stats


In [3]:
path = 'C:\\Parkinson\\processed'

In [4]:
def prepare_signal(path,hand,task,smooth,ordermin,ordermax,fs, graphics, Norm): #функция для построения сигналов

    data = json.load(open(path))

    frame=[]
    znach=[]
    if task==1:
        for i in range(len(data)):
            if hand in data[i].keys():
               
                sum_sqr = (float(data[i][hand]['FORE_TIP']['X1']) - float(data[i][hand]['THUMB_TIP']['X1'])) ** 2 \
                + (float(data[i][hand]['FORE_TIP']['Y1']) -float(data[i][hand]['THUMB_TIP']['Y1'])) ** 2 \
                + (float(data[i][hand]['FORE_TIP']['Z1']) -float(data[i][hand]['THUMB_TIP']['Z1'])) ** 2
                distance = math.sqrt(sum_sqr)
                znach.append(distance)
                frame.append(data[i]['frame'])

    if task == 2:
        for i in range(len(data)):
            if hand in data[i].keys():

                sum_sqr = (float(data[i][hand]['MIDDLE_TIP']['X1']) - float(data[i][hand]['CENTRE']['X'])) ** 2 \
                + (float(data[i][hand]['MIDDLE_TIP']['Y1']) -float(data[i][hand]['CENTRE']['Y'])) ** 2 \
                + (float(data[i][hand]['MIDDLE_TIP']['Z1']) -float(data[i][hand]['CENTRE']['Z'])) ** 2
                distance = math.sqrt(sum_sqr)
                znach.append(distance)
                frame.append(data[i]['frame'])

    x_c=[]
    y_c=[]

    if task == 3:
        for i in range(len(data)):
            if hand in data[i].keys():
                x_c.append(float(data[i][hand]['LITTLE_DIP']['X1']))
                y_c.append(float(data[i][hand]['LITTLE_DIP']['Y1']))
                
                znach.append(math.sqrt(float(data[i][hand]['LITTLE_DIP']['Z3'])**2\
                                       +float(data[i][hand]['LITTLE_DIP']['X3'])**2\
                                       +float(data[i][hand]['LITTLE_DIP']['Y3'])**2))
                

                frame.append(data[i]['frame'])

    filtered = lowess(znach, frame, is_sorted=True, frac=smooth, it=0)  # 0.02
   
        
    frame, value = np.array(filtered[:, 0]), np.array(filtered[:, 1])
    maxTemp = argrelmax(value, order=ordermax)
    minTemp = argrelmin(value, order=ordermin)
    mines = []
    maxes = []

    for maxi in maxTemp[0]:
        if value[maxi] > 0:
            maxes.append(maxi)

    for mini in minTemp[0]:
        if value[mini] > 0:
            mines.append(mini)
    if Norm:
        maxi = value.max()
        value =  value / maxi

        
    if graphics:
        plt.plot(frame, znach, 'r')
        plt.plot(frame, filtered[:, 1], 'b')
        plt.show()
        
        plt.plot(frame, value, '-')
        plt.plot(frame[maxes], value[maxes], 'ro')
        plt.plot(frame[mines], value[mines], 'bo')
        plt.show()
    time = frame/fs

    return frame, value, time, maxes, mines 




In [5]:
def amplitude_min_max(signal, maxi,mini): 
    return np.max(signal[maxi])-np.min(signal[mini]) #np.round(np.median(signal[maxi])-np.median(signal[mini]),3)

In [6]:
def amplitude_median(signal, maxi,mini): 
    return np.round(np.median(signal[maxi])-np.median(signal[mini]),3)

In [7]:
def frequency (fs, frames, maxi):
    time = len(frames)/fs
    return np.round(len(maxi)/time,3)

In [8]:
def add_to_task (task, stadia, patient, m, hand,  amp_min_max, amp_median , freq): #функция добаво
    if len(task) !=0:
        newrow = [patient + ' ' + m + ' ' + hand, amp_min_max, amp_median , freq, stadia[0]]
        task = np.vstack([task, newrow])
    else:
        task = np.array([patient + ' ' + m + ' ' + hand, amp_min_max, amp_median, freq, stadia[0]])
    return task

In [9]:
def patients (stadia, path, graph, f, stats, norm, task1, task2,task3, problems, inf):
    
    for patient in os.listdir(os.path.join(path,stadia)): #перебираем пациентов
            patient_path = os.path.join(path,stadia,patient)+'\\hand' # переходим в папку hand
            for record in (os.listdir(patient_path)): # проходим по каждой записи
                if record.find('json')>0 and  record.find('TR') < 0: # выбираем только json и исключаем записи тремора 
                
                    h= re.search('_[A-Z]_',record ).group(0)[1] # Определяем на какую руку было упражнение
                    if h == 'L':
                        hand = 'left hand'
                    else:
                        hand = 'right hand'

                    m = re.search('_[a-z]\d_',record ).group(0)[1:3] 
                    zadacha = re.search('\d_[A-Z]',record ).group(0)[0] # Определяем какое  было упражнение
                    
                    file = os.path.join(patient_path,record)
                    if inf:
                        print(record,patient, stadia)
                    frames, values,times, maxi,mini = prepare_signal(file,hand,int(zadacha), \
                                                              smooth=0.007,ordermin=15,
                                                                     ordermax=15, fs = f ,graphics=graph, Norm = norm)
                    if len(frames)==0:
                        problems.append(record) #если файл пустой, запишем его название 
                        
                    if stats and len(frames)>0:
                        amp_min_max = amplitude_min_max (values,maxi,mini)
                        amp_median = amplitude_median (values,maxi,mini)
                        freq = frequency (f, frames, maxi)
 
                    
                    if zadacha == '1':
                        task1 = add_to_task(task1, stadia, patient, m, hand, float( amp_min_max ),float( amp_median ), float(freq))
                       
                    if zadacha == '2':
                        task2 = add_to_task(task2, stadia, patient, m, hand, float( amp_min_max ),float( amp_median ), float(freq))
                        
                    if zadacha == '3':
                        task3 = add_to_task(task3, stadia, patient, m, hand,float( amp_min_max ),float( amp_median ), float(freq))
    return task1, task2,task3, problems

In [10]:
def signal_processing(path,stats = True, graph=False, fs = 120, Norm = True, inf = True ):
    task1 = []
    task2 = []
    task3 = []
    problems = []
    for stadia in os.listdir(path):
        if stadia == '0_stage':
            task1, task2, task3, problems = patients (stadia, path, graph, fs, stats, Norm, task1, task2, task3, problems, inf)
        if stadia == '1_stage':
            task1, task2, task3, problems = patients (stadia, path, graph, fs, stats, Norm, task1, task2, task3, problems, inf)
        if stadia == '2_stage':
            task1, task2, task3, problems = patients (stadia, path, graph, fs, stats, Norm, task1, task2, task3, problems,inf)
        if stadia == '3_stage':
            task1, task2, task3, problems = patients (stadia, path, graph, fs, stats, Norm,  task1, task2, task3, problems,inf)
    return task1, task2, task3, problems

In [11]:
a1,a2,a3,a4 = signal_processing(path, graph=False, inf = False, Norm = False)

c:\Users\Asus\anaconda3\envs\polguev\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Asus\anaconda3\envs\polguev\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [12]:
df_task1 = pd.DataFrame(a1, columns = ['Запись', 'Амплитуда_min_max', 'Амплитуда_median', 'Частота', 'Таргет'])
df_task1['Частота'] = df_task1['Частота'].astype(float)
df_task1['Амплитуда_min_max'] = df_task1['Амплитуда_min_max'].astype(float)
df_task1['Амплитуда_median'] = df_task1['Амплитуда_median'].astype(float)

df_task2 = pd.DataFrame(a2, columns = ['Запись', 'Амплитуда_min_max', 'Амплитуда_median', 'Частота', 'Таргет'])
df_task2['Частота'] = df_task2['Частота'].astype(float)
df_task2['Амплитуда_min_max'] = df_task2['Амплитуда_min_max'].astype(float)
df_task2['Амплитуда_median'] = df_task2['Амплитуда_median'].astype(float)

df_task3 = pd.DataFrame(a3, columns = ['Запись', 'Амплитуда_min_max', 'Амплитуда_median', 'Частота', 'Таргет'])
df_task3['Частота'] = df_task3['Частота'].astype(float)
df_task3['Амплитуда_min_max'] = df_task3['Амплитуда_min_max'].astype(float)
df_task3['Амплитуда_median'] = df_task3['Амплитуда_median'].astype(float)

In [13]:
df_task1

,Запись,Амплитуда_min_max,Амплитуда_median,Частота,Таргет
0,Patient12 m2 left hand,110.803242,73.403,4.413,0
1,Patient12 m2 right hand,115.329095,87.265,4.216,0
2,Patient13 m1 left hand,82.659496,54.958,3.792,0
3,Patient13 m2 left hand,90.443139,69.894,3.144,0
4,Patient13 m1 right hand,65.362022,26.710,5.227,0
...,...,...,...,...,...
87,Patient47 m2 right hand,16.286178,5.579,3.967,3
88,Patient51 m1 left hand,38.267873,27.567,4.469,3
89,Patient51 m2 left hand,86.028494,14.020,4.886,3
90,Patient51 m1 right hand,45.572900,28.037,4.165,3
